## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

## **2) Variables for Neural Networks & Data**

In [2]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0001)

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [3]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))
  
  model.summary()
  return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                        

## **4) Building Discriminator**

In [4]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


## **5) Connecting Neural Networks to build GAN**

In [5]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         362000    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 895,505
Trainable params: 360,464
Non-trainable params: 535,041
_________________________________________________________________


## **6) Outputting Images**


In [6]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [7]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  # X_train = np.expand_dims(X_train, axis=3)
  # print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    #Get Random Batch
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(30000, batch_size=64, save_interval=200)

11501568/11490434 [==============================] - 2s 0us/step
******* 0 [D loss: 0.749412, acc: 57.81%] [G loss: 0.772079]
0.00000001
saved
******* 1 [D loss: 0.383969, acc: 90.62%] [G loss: 0.755796]
******* 2 [D loss: 0.314284, acc: 91.41%] [G loss: 0.784572]
******* 3 [D loss: 0.330195, acc: 81.25%] [G loss: 0.799104]
******* 4 [D loss: 0.322730, acc: 82.81%] [G loss: 0.887687]
******* 5 [D loss: 0.310788, acc: 85.16%] [G loss: 0.907894]
******* 6 [D loss: 0.322648, acc: 79.69%] [G loss: 0.969125]
******* 7 [D loss: 0.296178, acc: 85.94%] [G loss: 1.035753]
******* 8 [D loss: 0.262530, acc: 93.75%] [G loss: 1.133568]
******* 9 [D loss: 0.243511, acc: 89.84%] [G loss: 1.237479]
******* 10 [D loss: 0.193707, acc: 96.09%] [G loss: 1.330214]
******* 11 [D loss: 0.193601, acc: 96.88%] [G loss: 1.465623]
******* 12 [D loss: 0.160655, acc: 97.66%] [G loss: 1.542353]
******* 13 [D loss: 0.133395, acc: 98.44%] [G loss: 1.728006]
******* 14 [D loss: 0.105807, acc: 99.22%] [G loss: 1.859360

******* 128 [D loss: 0.010922, acc: 100.00%] [G loss: 4.090446]
******* 129 [D loss: 0.011841, acc: 100.00%] [G loss: 4.142361]
******* 130 [D loss: 0.009743, acc: 100.00%] [G loss: 4.106730]
******* 131 [D loss: 0.010875, acc: 100.00%] [G loss: 4.126907]
******* 132 [D loss: 0.011294, acc: 100.00%] [G loss: 4.139738]
******* 133 [D loss: 0.009890, acc: 100.00%] [G loss: 4.170854]
******* 134 [D loss: 0.010610, acc: 100.00%] [G loss: 4.182239]
******* 135 [D loss: 0.013032, acc: 100.00%] [G loss: 4.107987]
******* 136 [D loss: 0.012013, acc: 100.00%] [G loss: 4.225892]
******* 137 [D loss: 0.010231, acc: 100.00%] [G loss: 4.215922]
******* 138 [D loss: 0.010500, acc: 100.00%] [G loss: 4.218530]
******* 139 [D loss: 0.009930, acc: 100.00%] [G loss: 4.168781]
******* 140 [D loss: 0.010229, acc: 100.00%] [G loss: 4.225330]
******* 141 [D loss: 0.012221, acc: 100.00%] [G loss: 4.212755]
******* 142 [D loss: 0.010851, acc: 100.00%] [G loss: 4.241899]
******* 143 [D loss: 0.009008, acc: 100.

******* 257 [D loss: 0.019167, acc: 100.00%] [G loss: 4.222802]
******* 258 [D loss: 0.029658, acc: 100.00%] [G loss: 4.445517]
******* 259 [D loss: 0.020214, acc: 100.00%] [G loss: 4.208968]
******* 260 [D loss: 0.017498, acc: 100.00%] [G loss: 4.339694]
******* 261 [D loss: 0.010806, acc: 100.00%] [G loss: 4.344544]
******* 262 [D loss: 0.018932, acc: 100.00%] [G loss: 4.277886]
******* 263 [D loss: 0.017267, acc: 100.00%] [G loss: 4.091115]
******* 264 [D loss: 0.028205, acc: 100.00%] [G loss: 4.248785]
******* 265 [D loss: 0.019001, acc: 100.00%] [G loss: 4.301277]
******* 266 [D loss: 0.023127, acc: 100.00%] [G loss: 4.345142]
******* 267 [D loss: 0.018475, acc: 100.00%] [G loss: 4.235542]
******* 268 [D loss: 0.018453, acc: 100.00%] [G loss: 4.212107]
******* 269 [D loss: 0.025796, acc: 100.00%] [G loss: 4.170849]
******* 270 [D loss: 0.020810, acc: 100.00%] [G loss: 4.277508]
******* 271 [D loss: 0.023270, acc: 100.00%] [G loss: 4.248735]
******* 272 [D loss: 0.019563, acc: 100.

******* 386 [D loss: 0.028675, acc: 100.00%] [G loss: 4.594582]
******* 387 [D loss: 0.132473, acc: 95.31%] [G loss: 3.798735]
******* 388 [D loss: 0.047139, acc: 99.22%] [G loss: 3.526187]
******* 389 [D loss: 0.054176, acc: 99.22%] [G loss: 3.161744]
******* 390 [D loss: 0.091370, acc: 97.66%] [G loss: 3.347083]
******* 391 [D loss: 0.055915, acc: 100.00%] [G loss: 3.623837]
******* 392 [D loss: 0.031024, acc: 100.00%] [G loss: 4.444917]
******* 393 [D loss: 0.066206, acc: 98.44%] [G loss: 4.176164]
******* 394 [D loss: 0.044632, acc: 100.00%] [G loss: 3.927414]
******* 395 [D loss: 0.067171, acc: 99.22%] [G loss: 3.785956]
******* 396 [D loss: 0.059568, acc: 99.22%] [G loss: 3.668272]
******* 397 [D loss: 0.060485, acc: 99.22%] [G loss: 3.697550]
******* 398 [D loss: 0.037477, acc: 100.00%] [G loss: 3.826905]
******* 399 [D loss: 0.035354, acc: 100.00%] [G loss: 4.156526]
******* 400 [D loss: 0.103890, acc: 96.88%] [G loss: 3.593200]
0.00000003
saved
******* 401 [D loss: 0.071110, a

******* 516 [D loss: 0.016429, acc: 100.00%] [G loss: 5.157812]
******* 517 [D loss: 0.026398, acc: 100.00%] [G loss: 4.845595]
******* 518 [D loss: 0.018287, acc: 100.00%] [G loss: 4.510298]
******* 519 [D loss: 0.019523, acc: 100.00%] [G loss: 4.778898]
******* 520 [D loss: 0.020864, acc: 100.00%] [G loss: 4.570711]
******* 521 [D loss: 0.020695, acc: 100.00%] [G loss: 4.845969]
******* 522 [D loss: 0.017793, acc: 100.00%] [G loss: 4.968638]
******* 523 [D loss: 0.030228, acc: 99.22%] [G loss: 4.746571]
******* 524 [D loss: 0.030123, acc: 100.00%] [G loss: 4.504227]
******* 525 [D loss: 0.023208, acc: 100.00%] [G loss: 4.404553]
******* 526 [D loss: 0.037853, acc: 99.22%] [G loss: 4.200606]
******* 527 [D loss: 0.019003, acc: 100.00%] [G loss: 4.632286]
******* 528 [D loss: 0.037968, acc: 99.22%] [G loss: 4.902267]
******* 529 [D loss: 0.039319, acc: 99.22%] [G loss: 4.580090]
******* 530 [D loss: 0.020325, acc: 100.00%] [G loss: 4.025679]
******* 531 [D loss: 0.021868, acc: 100.00%]

...

******* 29457 [D loss: 0.433613, acc: 82.03%] [G loss: 1.436863]
******* 29458 [D loss: 0.448709, acc: 80.47%] [G loss: 1.750649]
******* 29459 [D loss: 0.563205, acc: 71.09%] [G loss: 1.853269]
******* 29460 [D loss: 0.603212, acc: 65.62%] [G loss: 1.606092]
******* 29461 [D loss: 0.611663, acc: 72.66%] [G loss: 1.319280]
******* 29462 [D loss: 0.503414, acc: 75.00%] [G loss: 1.403164]
******* 29463 [D loss: 0.488292, acc: 77.34%] [G loss: 1.450997]
******* 29464 [D loss: 0.527823, acc: 72.66%] [G loss: 1.815621]
******* 29465 [D loss: 0.534190, acc: 73.44%] [G loss: 1.492958]
******* 29466 [D loss: 0.566847, acc: 72.66%] [G loss: 1.521413]
******* 29467 [D loss: 0.458922, acc: 79.69%] [G loss: 1.608027]
******* 29468 [D loss: 0.573052, acc: 68.75%] [G loss: 1.590990]
******* 29469 [D loss: 0.651201, acc: 67.97%] [G loss: 1.504382]
******* 29470 [D loss: 0.586283, acc: 66.41%] [G loss: 1.400122]
******* 29471 [D loss: 0.548929, acc: 71.88%] [G loss: 1.392255]
******* 29472 [D loss: 0.

******* 29585 [D loss: 0.433736, acc: 82.03%] [G loss: 1.649371]
******* 29586 [D loss: 0.505943, acc: 77.34%] [G loss: 1.682346]
******* 29587 [D loss: 0.539370, acc: 69.53%] [G loss: 1.770293]
******* 29588 [D loss: 0.522558, acc: 77.34%] [G loss: 1.366376]
******* 29589 [D loss: 0.490683, acc: 77.34%] [G loss: 1.631962]
******* 29590 [D loss: 0.516459, acc: 75.78%] [G loss: 1.540405]
******* 29591 [D loss: 0.550210, acc: 74.22%] [G loss: 1.474917]
******* 29592 [D loss: 0.446590, acc: 79.69%] [G loss: 1.805361]
******* 29593 [D loss: 0.395267, acc: 82.81%] [G loss: 1.977364]
******* 29594 [D loss: 0.459933, acc: 79.69%] [G loss: 1.746897]
******* 29595 [D loss: 0.506001, acc: 76.56%] [G loss: 1.541263]
******* 29596 [D loss: 0.483225, acc: 80.47%] [G loss: 1.583420]
******* 29597 [D loss: 0.454139, acc: 78.12%] [G loss: 1.501421]
******* 29598 [D loss: 0.470396, acc: 79.69%] [G loss: 1.494484]
******* 29599 [D loss: 0.553104, acc: 72.66%] [G loss: 1.574228]
******* 29600 [D loss: 0.

******* 29712 [D loss: 0.501928, acc: 75.00%] [G loss: 1.461683]
******* 29713 [D loss: 0.467691, acc: 78.12%] [G loss: 1.671201]
******* 29714 [D loss: 0.541617, acc: 73.44%] [G loss: 1.837591]
******* 29715 [D loss: 0.420593, acc: 82.03%] [G loss: 1.688597]
******* 29716 [D loss: 0.427945, acc: 82.81%] [G loss: 1.714015]
******* 29717 [D loss: 0.451709, acc: 77.34%] [G loss: 1.400429]
******* 29718 [D loss: 0.436059, acc: 77.34%] [G loss: 1.491051]
******* 29719 [D loss: 0.435111, acc: 84.38%] [G loss: 1.820446]
******* 29720 [D loss: 0.564105, acc: 71.88%] [G loss: 1.653080]
******* 29721 [D loss: 0.448214, acc: 78.91%] [G loss: 1.548322]
******* 29722 [D loss: 0.491154, acc: 75.00%] [G loss: 1.561452]
******* 29723 [D loss: 0.490084, acc: 75.00%] [G loss: 1.550764]
******* 29724 [D loss: 0.507056, acc: 80.47%] [G loss: 1.531301]
******* 29725 [D loss: 0.462408, acc: 75.78%] [G loss: 1.641595]
******* 29726 [D loss: 0.535228, acc: 78.12%] [G loss: 1.697234]
******* 29727 [D loss: 0.

******* 29837 [D loss: 0.559833, acc: 73.44%] [G loss: 1.574160]
******* 29838 [D loss: 0.515623, acc: 76.56%] [G loss: 1.340094]
******* 29839 [D loss: 0.499574, acc: 73.44%] [G loss: 1.621602]
******* 29840 [D loss: 0.615573, acc: 65.62%] [G loss: 1.627457]
******* 29841 [D loss: 0.502688, acc: 79.69%] [G loss: 1.723626]
******* 29842 [D loss: 0.586132, acc: 73.44%] [G loss: 1.444160]
******* 29843 [D loss: 0.488297, acc: 72.66%] [G loss: 1.248051]
******* 29844 [D loss: 0.510663, acc: 75.00%] [G loss: 1.203056]
******* 29845 [D loss: 0.564061, acc: 71.09%] [G loss: 1.828155]
******* 29846 [D loss: 0.626469, acc: 68.75%] [G loss: 1.581042]
******* 29847 [D loss: 0.522664, acc: 78.91%] [G loss: 1.536402]
******* 29848 [D loss: 0.417807, acc: 82.03%] [G loss: 1.680922]
******* 29849 [D loss: 0.517918, acc: 76.56%] [G loss: 1.394907]
******* 29850 [D loss: 0.551177, acc: 74.22%] [G loss: 1.445169]
******* 29851 [D loss: 0.492957, acc: 75.00%] [G loss: 1.400632]
******* 29852 [D loss: 0.

******* 29964 [D loss: 0.620632, acc: 67.19%] [G loss: 1.456533]
******* 29965 [D loss: 0.570491, acc: 67.19%] [G loss: 1.559885]
******* 29966 [D loss: 0.469267, acc: 78.12%] [G loss: 1.477344]
******* 29967 [D loss: 0.581427, acc: 71.88%] [G loss: 1.395022]
******* 29968 [D loss: 0.483479, acc: 78.12%] [G loss: 1.505396]
******* 29969 [D loss: 0.459924, acc: 80.47%] [G loss: 1.419604]
******* 29970 [D loss: 0.518491, acc: 75.78%] [G loss: 1.504939]
******* 29971 [D loss: 0.555364, acc: 74.22%] [G loss: 1.249326]
******* 29972 [D loss: 0.613799, acc: 71.88%] [G loss: 1.497057]
******* 29973 [D loss: 0.505088, acc: 80.47%] [G loss: 1.719280]
******* 29974 [D loss: 0.431681, acc: 81.25%] [G loss: 1.774139]
******* 29975 [D loss: 0.482397, acc: 76.56%] [G loss: 1.610176]
******* 29976 [D loss: 0.526395, acc: 76.56%] [G loss: 1.421117]
******* 29977 [D loss: 0.510589, acc: 73.44%] [G loss: 1.267298]
******* 29978 [D loss: 0.560746, acc: 74.22%] [G loss: 1.605478]
******* 29979 [D loss: 0.

### **8) Making GIF**

In [8]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)